<a href="https://colab.research.google.com/github/xinyueli2896/raptranscription/blob/main/loader_prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create an audiofolder containing:
1.   train: segmented mp3 files for training
2.   test: segmented mp3 files for training
3.   metadata.csv:  path to the mp3 files and label for the each file(lyrics)  





In [ ]:
!pip install faker
!pip install datasets
!pip install pydub
!pip install datasets

In [ ]:
import pandas as pd
import uuid
import random
from faker import Faker
import datetime
import os
from datasets import Dataset
from datasets import DatasetDict
from pydub import AudioSegment
import re
import csv

In [ ]:
num_songs = 187

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
dataset_path = "/content/drive/MyDrive/data"
songs_path = "/content/drive/MyDrive/data/songs"
lyrics_path = "/content/drive/MyDrive/data/lyrics"

Mounted at /content/drive/


In [ ]:
# @title Segment songs into chunks according to the timestamps in .lrc filesave them to individual mp3 files, store mp3 file path and lyrics into an array of size number_chunks*2
def parse_lrc(lrc_content):
    """
    Parse the LRC file content and extract timestamps and lyrics.
    Returns a list of tuples (start_time, end_time, lyric).
    """
    timestamps = []
    lyrics = []
    for line in lrc_content.split('\n'):
        match = re.match(r'\[(\d+):(\d+\.\d+)\](.*)', line)
        if match:
            minutes, seconds, lyric = match.groups()
            time = int(minutes) * 60 * 1000 + int(float(seconds) * 1000)  # Convert to milliseconds
            timestamps.append(time)
            lyrics.append(lyric)

    # Create tuples of (start_time, end_time, lyric)
    intervals = [(timestamps[i], timestamps[i+1], lyrics[i]) for i in range(len(timestamps)-1)]
    return intervals


output_folder_songs = os.path.join(dataset_path, 'segmented_songs')
if not os.path.exists(output_folder_songs):
    os.makedirs(output_folder_songs)

data = []

n=0
delimiters = "-(."
pattern = r'[' + delimiters + r']\s*'
for filename in os.listdir(songs_path):
  if filename.endswith('.flac'):

    audio = AudioSegment.from_file(os.path.join(songs_path,filename), format="flac") # read audio
    name = re.split(pattern, filename)[0]
    for filename1 in os.listdir(lyrics_path): # search for lyrics file
      if filename1.startswith(name):
        with open(os.path.join(lyrics_path,filename1), 'r') as file:
          lrc_content = file.read()
        intervals = parse_lrc(lrc_content) # parse lyrics

        for i, (start, end, lyric) in enumerate(intervals):
          if i<3:
            continue
          chunk = audio[start:end]
          chunk.export(f"{output_folder_songs}/chunk_{n}.mp3", format="mp3")
          data.append([f"data/chunk_{n}.mp3",lyric])
          n = n+1

In [ ]:
data.insert(0, ['file_name','lyrics']

In [ ]:
len(data)

15990

In [ ]:
# @title
import re
delimiters = "-(."
for filename in os.listdir(songs_path):
  pattern = r'[' + delimiters + r']\s*'
  words = re.split(pattern, filename)
  print(words)

for filename in os.listdir(lyrics_path):
  pattern = r'[' + delimiters + r']\s*'
  words = re.split(pattern, filename)
  print(words)

In [ ]:
# @title Store your array into a csv
file_path = 'metadata.csv'

# Writing to a CSV file
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    for row in data:
        writer.writerow(row)

print("Data written to", file_path)


Finally, manually split the files in the segmented songs folder to train folder and test folder and make changes to your csv to reflect the change in file paths.